# HackOnData.com

## Exercise #3 - Log Analysis

In [2]:
%python

proj_folder = "/mnt/E3"
filename = 'TQ_web_analytics_de_identified.locreag'

dbutils.fs.mkdirs(proj_folder)

In [3]:
%%bash
wget https://s3-us-west-2.amazonaws.com/tq-opendata/TQ_web_analytics_de_identified.log

In [4]:
import operator
import os
import random
import re

from pyspark.sql import functions, Row, types

from httpagentparser import simple_detect
from ua_parser.user_agent_parser import Parse


dbutils.fs.cp("file:/databricks/driver/%s" % filename, proj_folder)
logfile = sc.textFile(os.path.join(proj_folder, filename), 8)

## TQ Log:
The following fields are included:

```
 log>
 |-- action: string (nullable = true) | Action
 |-- obj_id: string (nullable = true) | Object Id
 |-- obj_type: string (nullable = true) | Object Type
 |-- timestamp: long (nullable = true) | Time Stamp in msec
 |-- type: string (nullable = true) | Type
 |-- ua: string (nullable = true) | User Agent
 |-- uuid: string (nullable = true) | UUID
```

Example:

| action | obj_id | obj_type |	timestamp |	type |	ua |	uuid |
|-------------|
|hover |	Accommodation and Food Services	| main_category_title |	1467962138212	| Event	| Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36	| 83e28957-3a80-4d3c-b189-339fe0d66c4b|


The log corresponds to front-end events for TranQuant.

### Text parsing
Read each line of the log file as text. Write a regular expression to parse the first 3 fields (action, obj_id, and obj_type), ignore the remaining fields for now. 

Hint: Similar to `def parseLogs():` in the Lab

In [7]:
def parse_serialized_json(logline):
    pattern = ('(?P<obj_type>(?<=obj_type":")[^\\"]+(?="))(?:.*)'
             '(?P<obj_id>(?<=obj_id":")[^\\"]+(?="))(?:.*)'
             '(?P<action>(?<="action":")[^\\"]+(?="))')
    match = re.search(pattern, logline)
    if not match: return
    print match.groups()
    return Row(
        obj_type      = match.group(1),
        obj_id        = match.group(2),
        action        = match.group(3))

What are the most popular (top 6) `obj_id`? present the results in a plot.

In [9]:
rows_rdd = logfile.map(parse_serialized_json)

obj_id_by_popularity = (rows_rdd
                        .map(lambda r: (r.obj_id, 1))
                        .reduceByKey(operator.add))

# visualize
display(sqlContext.createDataFrame(
    obj_id_by_popularity.top(
        6, 
        key=lambda (id, c): c), 
    ['Object ID', 'Frequency']))

In [10]:
log = sqlContext.read.json(os.path.join(proj_folder, filename))
# log.printSchema()

In [11]:
log.select('obj_id', 'obj_type', 'action').show(10, truncate=False)

In [12]:
display(log)

In [13]:
log.describe(['timestamp']).show()

### User-Agent Fields

In [15]:
# visualize most-frequent User-Agent strings (probably from the same machine)

def parse_ua(logline):
    pattern = '(?P<ua>(?<=ua":")[^\\"]+(?="))'
    match = re.search(pattern, logline)
    if not match: return
    return (match.group(1), 1)

uaRDD = (logfile
         .map(parse_ua)
         .reduceByKey(operator.add)

display(sqlContext.createDataFrame(
    data=uaRDD.takeOrdered(
      10, 
      lambda (ua, cnt): -1 * cnt), 
    schema=['User Agent', 'Frequency']))

In [16]:
#parse User-Agent

uaDF = log.map(lambda r: Row(
    ts        = r.timestamp,
    ua_client = simple_detect(r.ua)[1],  # naive
    ua        = Parse(r.ua), 
    )).toDF().cache()

display(uaDF)

In [17]:
# visualize Client Popularity (Deterministic mode)

all_clientsDF = sqlContext.createDataFrame(
  uaDF
    .map(lambda r: (r.ua['user_agent']['family'], 1))
    .reduceByKey(operator.add)
    .takeOrdered(20, lambda (cl, c): -1 * c), 
  schema=['Browser', 'Hits'])

display(all_clientsDF)

### Traffic over typical 24 hours

In [19]:
# visualize resource popularity across 24 hours

# https://spark.apache.org/docs/1.5.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext.createDataFrame
schema = [ 
  ['Hour'],
  types.StructType([
    types.StructField("Hour", types.IntegerType(), nullable=False),
  ])]

display(sqlContext.createDataFrame(
    uaDF.map(lambda r: Row(Hour=(r.ts // 3600000) % 24)), 
    random.choice(schema)))

# alternative approach (and if ts was converted into types.TimestampType().fromInternal(original_timestamp*1000))
# display(uaDF.select(functions.hour('ts')).alias('Hour')


Repeat the same exercise for the the top 3 browsers, do you find any interesting results?

In [21]:
popularityDF = (uaDF
    .rdd  
    .map(lambda r: (r.ua['user_agent']['family'], 1))
    .reduceByKey(operator.add)
    .toDF(['BrowserFamily', 'Hits'])
    .orderBy('Hits', ascending=False))

# filter top 3 browsers
display(popularityDF.limit(3))

In [22]:
overview = (uaDF
  .map(lambda r: (
      (r.ua['user_agent']['family'], (r.ts // 3600000) % 24), 
      1))
  .reduceByKey(operator.add)
  # .sortByKey()  # equivalent to orderBy when presenting
  .map(lambda ((f, h), c): Row(
      BrowserFamily=f, 
      Hour=h,
      Freqeuncy=c))
  .toDF())                         # back to DF 

display(overview.orderBy(['Hour', 'BrowserFamily']))

In [23]:
# let's make a list of cleints, (pre)sorted by popularity
clients_by_popularity = popularityDF.map(lambda r: r.BrowserFamily).collect()

In [24]:
# for the top 3, we will use previously derived `popularityDF`
leader_stats = overview[overview.BrowserFamily.isin(clients_by_popularity[:3])]

display(top_browser_stats.orderBy(['Hour', 'BrowserFamily']))

In [25]:
# let's also see the rest of the clients besides the leaders
less_popular_browser_stats = overview[overview.BrowserFamily.isin(clients_by_popularity[3:])]

display(less_popular_browser_stats.orderBy(['Hour', 'BrowserFamily']))

In [26]:
chrome, firefox, safari = zip(*most_popular_browsersRDD)[0]
stats = lambda browser: (
  uaDF
  .where(uaDF.ua['user_agent']['family'] == browser)
  .select(hour('ts')).alias('Hour'))


#### Chrome :::::::::::::::
Usage of **Chrome** browser family over typical 24 hours course

In [28]:
display(stats(chrome))

#### Firefox :::::::::::::::
Usage of **Firefox** browser family over typical 24 hours course

In [30]:
display(stats(firefox))

#### Safari :::::::::::::::
Usage of **Safari** browser family over typical 24 hours course

In [32]:
display(stats(safari))